## _*Using Qiskit Aqua for max-cut problems*_

This Qiskit Aqua Optimization notebook demonstrates how to use the VQE quantum algorithm to compute the max cut of a given graph. 



In [1]:
import numpy as np
from qiskit import Aer
from qiskit.optimization.ising import max_cut
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import ExactEigensolver, VQE,QAOA
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.optimization.ising.common import parse_gset_format, random_graph, sample_most_likely

Here an Operator instance is created for our Hamiltonian. In this case the paulis are from an Ising Hamiltonian translated from the max-cut problem. We load a small sample instance of the max-cut problem.

In [2]:
w=np.array([[0,1,1],[1,0,0],[1,0,0]])
print(w)

[[0 1 1]
 [1 0 0]
 [1 0 0]]


We also offer a function to generate a random graph as a input.

In [3]:
qubitOp, offset = max_cut.get_operator(w)
qubitOp.paulis

[[(0.5+0j), Pauli(z=[True, True, False], x=[False, False, False])],
 [(0.5+0j), Pauli(z=[True, False, True], x=[False, False, False])]]

## _*Classical method*_

In [4]:
result = ExactEigensolver(qubitOp).run()
x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('max-cut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

energy: -1.0
max-cut objective: -2.0
solution: [0. 1. 1.]
solution objective: 2.0


## _*VQE method*_

In [5]:
vqe = VQE(qubitOp,
          RYRZ(qubitOp.num_qubits, depth=3, entanglement='linear'),
          L_BFGS_B(maxfun=6000))
result = vqe.run(QuantumInstance(Aer.get_backend('statevector_simulator')))

x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('max-cut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

/home/ibmqopen/anaconda3/envs/Qdemo/lib/python3.7/site-packages/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/home/ibmqopen/anaconda3/envs/Qdemo/lib/python3.7/site-packages/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/home/ibmqopen/anaconda3/envs/Qdemo/lib/python3.7/site-packages/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You sh

energy: -0.9999999998613587
time: 15.44348406791687
max-cut objective: -1.9999999998613587
solution: [0. 1. 1.]
solution objective: 2.0


## _*QAOA method*_

In [6]:
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import SPSA
spsa = SPSA(max_trials=250)
qaoa = QAOA(qubitOp, spsa, p=3)
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024)
result = qaoa.run(quantum_instance)
x = sample_most_likely(result['eigvecs'][0])


/home/ibmqopen/anaconda3/envs/Qdemo/lib/python3.7/site-packages/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/home/ibmqopen/anaconda3/envs/Qdemo/lib/python3.7/site-packages/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/home/ibmqopen/anaconda3/envs/Qdemo/lib/python3.7/site-packages/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You sh

In [7]:
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('max-cut objective:', result['energy'] + offset)
print('solution:', max_cut.get_graph_solution(x))
print('solution objective:', max_cut.max_cut_value(x, w))

energy: 1.0
time: 9.600014686584473
max-cut objective: 0.0
solution: [0 0 0]
solution objective: 0


In [8]:
result['min_vector']

{'111': 882, '000': 142}